In [ ]:
# 🚀 Auto-setup: installs deps + configures CFBD access
%run ./_auto_setup.py


In [ ]:
# 03_metrics_comparison.ipynb
# 📊 Comparing Team Efficiency Metrics

# 🛠 Requirements:
# - pandas, matplotlib, seaborn (install via `pip install pandas matplotlib seaborn`)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from pathlib import Path

# Import starter pack configuration system
_config_dir = Path().resolve() / "config"
if str(_config_dir.parent) not in sys.path:
    sys.path.insert(0, str(_config_dir.parent))
from config.data_config import get_starter_pack_config

# Get configuration
config = get_starter_pack_config()
current_year = config.current_year
DATA_DIR = str(config.data_dir)

# Configure plots
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = [15,8]


In [ ]:
# 📂 Load team-level advanced season stats for a recent year

stats_path = os.path.join(DATA_DIR, "advanced_season_stats", f"{current_year}.csv")
stats = pd.read_csv(stats_path)

# Filter to FBS teams
teams = stats[stats['conference'].notnull()].copy()

# Drop rows with missing metric values
teams = teams.dropna(subset=[
    'offense_ppa', 'offense_successRate', 'offense_explosiveness',
    'defense_ppa', 'defense_successRate', 'defense_explosiveness'
])

print(f"{len(teams)} teams with complete data")


In [ ]:
# 🧠 Let's look at correlations between key offensive metrics

off_metrics = teams[['offense_ppa', 'offense_successRate', 'offense_explosiveness']]
off_corr = off_metrics.corr()

sns.heatmap(off_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Offensive Metrics")
plt.show()

In [ ]:
# 🔍 EPA vs. Success Rate (Offense)
sns.scatterplot(
    data=teams,
    x='offense_successRate',
    y='offense_ppa',
    hue='conference',
    alpha=0.8
)
plt.title("Offensive EPA vs. Success Rate")
plt.xlabel("Success Rate")
plt.ylabel("EPA per Play")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()


In [ ]:
# 🎯 Highlight teams with high explosiveness but low success rate

threshold_explosive = teams['offense_explosiveness'].quantile(0.85)
threshold_success = teams['offense_successRate'].quantile(0.40)

boom_bust = teams[
    (teams['offense_explosiveness'] >= threshold_explosive) &
    (teams['offense_successRate'] <= threshold_success)
]

boom_bust[['team', 'conference', 'offense_ppa', 'offense_successRate', 'offense_explosiveness']]

In [ ]:
# 📊 Defensive profile: PPA vs. Success Rate

sns.scatterplot(
    data=teams,
    x='defense_successRate',
    y='defense_ppa',
    hue='conference',
    alpha=0.8
)
plt.title("Defensive EPA vs. Success Rate")
plt.xlabel("Defensive Success Rate (lower is better)")
plt.ylabel("Defensive EPA per Play (lower is better)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()


In [ ]:
# Let's repeat the process, adding some additional metrics

# Filter to FBS teams
teams = stats[stats['conference'].notnull()].copy()

# Drop rows with missing metric values
teams = teams.dropna(subset=[
    'offense_passingPlays_ppa', 'offense_rushingPlays_ppa', 'offense_havoc_frontSeven', 'offense_successRate', 'offense_explosiveness',
    'defense_passingPlays_ppa', 'defense_rushingPlays_ppa', 'defense_havoc_frontSeven', 'defense_successRate', 'defense_explosiveness'
])

print(f"{len(teams)} teams with complete data")

In [ ]:
# Re-run the correlation analysis with new metrics

off_metrics = teams[[ 'offense_passingPlays_ppa', 'offense_rushingPlays_ppa', 'offense_havoc_frontSeven', 'offense_successRate', 'offense_explosiveness']]
off_corr = off_metrics.corr()

sns.heatmap(off_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Offensive Metrics")
plt.show()

In [ ]:
# Repeat again for defensive metrics

off_metrics = teams[['defense_passingPlays_ppa', 'defense_rushingPlays_ppa', 'defense_havoc_frontSeven', 'defense_successRate', 'defense_explosiveness']]
off_corr = off_metrics.corr()

sns.heatmap(off_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Between Offensive Metrics")
plt.show()

In [ ]:
# ✅ Summary

# In this notebook, you:
# - Compared offensive and defensive metrics
# - Identified correlations and outliers
# - Found "boom-or-bust" style offenses
# - Visualized efficiency tradeoffs

# 🧪 Try This:
# - Build clusters of teams based on their profiles (KMeans, t-SNE)
# - Create matchup visualizations using opponent stats
# - Track metric evolution over time using weekly data (advanced_game_stats/)